In [1]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict
import tqdm
import matplotlib.pyplot as plt
from plotting import plot_3d, plot_2d
from tf2kfac import tf2KFAC
%matplotlib widget

### First lets compare the pure loss computation

In [2]:
input_data = tf.convert_to_tensor(np.load('data/input.npy'), dtype=tf.float32)
weights = tf.convert_to_tensor(np.load('data/weights.npy'), dtype=tf.float32)
target = tf.convert_to_tensor(np.load('data/target.npy'), dtype=tf.float32)

In [3]:
# need a softmax layer to define the output distribution p(y | x) to compute the fisher

with tf.GradientTape() as g:
    g.watch(weights)
    loss = tf.reduce_mean(((input_data@weights) - target)**2)  
print('Loss', loss)
print('Gradients', g.gradient(loss, weights))

Loss tf.Tensor(5.943384, shape=(), dtype=float32)
Gradients tf.Tensor(
[[ 0.47266078]
 [-5.364277  ]], shape=(2, 1), dtype=float32)


### KFAC

In [5]:
input_data = tf.convert_to_tensor(np.load('data/input.npy'), dtype=tf.float32)
weights = tf.convert_to_tensor(np.load('data/weights.npy'), dtype=tf.float32)
target = tf.convert_to_tensor(np.load('data/target.npy'), dtype=tf.float32)
kfac = tf2KFAC(lr0=0.001,
               cov_weight=1.,
               cov_moving_weight=0.95,
               damping=0.001,
               conv_approx='mg',
               damping_method='factored_tikhonov',
               ft_method='original')

history = []
for i in range(100):
    activations = input_data
    with tf.GradientTape(persistent=True) as g:
        g.watch(weights)
        prediction = input_data@weights
        loss = tf.reduce_mean((prediction - target)**2)
    sensitivities = g.gradient(loss, prediction)
    grads = g.gradient(loss, weights)
    ng = kfac.compute_updates([activations], [sensitivities], [grads], i)
    history.append(np.concatenate([weights.numpy().reshape(-1), loss.numpy()[None]]))
    weights = weights + ng[0]
    if i % 10 == 0:
        print(i, 'Loss', loss)
history = np.array(history, dtype=np.float32)

Factored Tikhonov damping
0 Loss tf.Tensor(5.943384, shape=(), dtype=float32)
10 Loss tf.Tensor(4.1790986, shape=(), dtype=float32)
20 Loss tf.Tensor(2.651995, shape=(), dtype=float32)
30 Loss tf.Tensor(1.4955992, shape=(), dtype=float32)
40 Loss tf.Tensor(0.8904588, shape=(), dtype=float32)
50 Loss tf.Tensor(0.7383152, shape=(), dtype=float32)
60 Loss tf.Tensor(0.7188817, shape=(), dtype=float32)
70 Loss tf.Tensor(0.7168057, shape=(), dtype=float32)
80 Loss tf.Tensor(0.71657205, shape=(), dtype=float32)
90 Loss tf.Tensor(0.7165429, shape=(), dtype=float32)


In [6]:
plt.close()
plot_3d(history, input_data, target)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.close()
plt.title('TF2 KFAC')
plot_2d(history, input_data, target, (-0.5, 1, -3, 1))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …